In [1]:
import numpy as np
np.random.seed(42)
import matplotlib as mpl
from matplotlib import pyplot as plt

In [2]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('MNIST_784', version=1, cache=True)
mnist.target = mnist.target.astype(np.int8)

In [8]:
X, y = mnist["data"], mnist["target"]
X.shape, y.shape

((70000, 784), (70000,))

In [33]:
split_index = 60000
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

TypeError: list indices must be integers or slices, not tuple